In [5]:
import pandas as pd
import zipfile
import os
import time
from tqdm import tqdm
import shutil
import math
import numpy as np

In [6]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [7]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [8]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

# Sliding Windows Features

In [9]:
files_add=find_the_way('./train-test','FE')
files_add

['./train-test\\Active Wiretap_FE_test.csv',
 './train-test\\Active Wiretap_FE_train.csv',
 './train-test\\ARP MitM_FE_test.csv',
 './train-test\\ARP MitM_FE_train.csv',
 './train-test\\Fuzzing_FE_test.csv',
 './train-test\\Fuzzing_FE_train.csv',
 './train-test\\Mirai_FE_test.csv',
 './train-test\\Mirai_FE_train.csv',
 './train-test\\OS Scan_FE_test.csv',
 './train-test\\OS Scan_FE_train.csv',
 './train-test\\SSDP Flood_FE_test.csv',
 './train-test\\SSDP Flood_FE_train.csv',
 './train-test\\SSL Renegotiation_FE_test.csv',
 './train-test\\SSL Renegotiation_FE_train.csv',
 './train-test\\SYN DoS_FE_test.csv',
 './train-test\\SYN DoS_FE_train.csv',
 './train-test\\Video Injection_FE_test.csv',
 './train-test\\Video Injection_FE_train.csv']

In [17]:
def diff(x):
    x = pd.Series(x)
    result=x.diff()
    try:
        result=result.fillna(result[1])
    except:result=result.fillna(0)
    return result
def mean(x):
    x = pd.Series(x)
    result=x.expanding(min_periods=1).mean()
    return result
def std(x):
    x = pd.Series(x)
    result=x.expanding(min_periods=1).std()
    return result
def sum_of(x):
    x = pd.Series(x)
    result=x.expanding(min_periods=1).sum()
    return result
def flagsum(x):
    total_f=0
    result=[]
    for i in x:
        if i==1:
            total_f+=1
        result.append(total_f)
    return result
def portsum(x):
    total_ports=[]
    result=[]
    for i in x:
        if i not in total_ports:
            total_ports.append(i)
        result.append(len(total_ports))
    return result     

In [18]:
n=['ts',"WS_src","WS_dst",
 'pck_size',
 #'Ether_dst',
 #'Ether_src','IP_src',
 #'IP_dst',
   'UDP_sport',
 'UDP_dport','TCP_sport',
 'TCP_dport',
#'IP_src',
 #'IP_dst', 
'TCP_FIN',
 'TCP_SYN',
 'TCP_RST',
 'TCP_PSH',
 'TCP_ACK',
 'TCP_URG',
 'TCP_ECE',
 'TCP_CWR',
 'TCP_window',"payload_bytes","entropy","Label"]

In [19]:
outputfolder="EW"
folder(outputfolder)


for j in files_add:
    print(j[13:])
    df=pd.read_csv(j)#,usecols=n)

    df["sport"]=df["UDP_sport"]+df["TCP_sport"]
    df["dport"]=df["UDP_dport"]+df["TCP_dport"]

    del df["UDP_sport"]
    del df["TCP_sport"]
    del df["UDP_dport"]
    del df["TCP_dport"]

    df.to_csv("temp.csv")


    df=pd.read_csv("temp.csv")
    #df["ID"] = df["Ether_src"] +"@" +df["IP_src"] +"====>"+df["Ether_dst"]+ "@"+ df["IP_dst"]
    df["ID"] = df["WS_src"]+"=>"+df["WS_dst"]
    #del df["Ether_src"] 
    #del df["IP_src"] 
    #del df["Ether_dst"] 
    #del df["IP_dst"]
    del df["WS_src"] 
    del df["WS_dst"]

    cols=[ 'TCP_FIN',
 'TCP_SYN',
 'TCP_RST',
 'TCP_PSH',
 'TCP_ACK',
 'TCP_URG',
 'TCP_ECE',
 'TCP_CWR', 'Label', 'ID', 'pck_size_diff', 'pck_size_mean',
           'pck_size_std',  'ts_diff', 'ts_mean', 'ts_std',
            'TCP_window_diff', 'TCP_window_mean', 'TCP_window_std',"payload_bytes_diff","payload_bytes_mean","payload_bytes_std",
            'entropy_diff',
     'entropy_mean',
     'entropy_std']



    IDS=sorted(list(df[df.columns[-1]].unique()))
    func=[diff,mean,std] #functions
    func_name=["diff","mean","std","sum_of"]
    f_list=["pck_size","ts",'TCP_window',"payload_bytes","entropy"]#, "dport","sport"     'TCP_SYN', 'TCP_ACK'    'TCP_FIN', 'TCP_SYN', 'TCP_RST', 'TCP_PSH', 'TCP_ACK', 'TCP_URG', 'TCP_ECE', 'TCP_CWR',  
    fark=['pck_size_diff',
     'pck_size_mean',
     'pck_size_std','pck_size_sum_of',
     'ts_diff',
     'ts_mean',
     'ts_std','ts_sum_of',
     'TCP_window_diff',
     'TCP_window_mean',
     'TCP_window_std','TCP_window_sum_of',
      "payload_bytes_diff","payload_bytes_mean","payload_bytes_std",'payload_bytes_sum_of',
     'entropy_diff',
     'entropy_mean',
     'entropy_std','entropy_sum_of']
    flag=1
    for i in tqdm(IDS):
        smaller = df[df["ID"]==i]
        smaller=smaller.reset_index()
        del smaller["index"]


        smaller["ts"]=np.array(diff(smaller["ts"].values))
        for ii in f_list:
            for jjj,iii in enumerate (func):
                name=str(ii)+"_"+str(func_name[jjj])
                smaller[name]=iii(smaller[ii])

        smaller["dport_sum"]=portsum(smaller["dport"].values)
        smaller["sport_sum"] =portsum(smaller["sport"].values)
        smaller['TCP_FIN_sum'  ]=flagsum (smaller[ 'TCP_FIN'  ].values)
        smaller['TCP_SYN_sum'  ]=flagsum (smaller['TCP_SYN'  ].values)
        smaller['TCP_RST_sum'  ]=flagsum (smaller['TCP_RST'  ].values)
        smaller['TCP_PSH_sum'  ]=flagsum (smaller['TCP_PSH'  ].values)
        smaller['TCP_ACK_sum'  ]=flagsum (smaller['TCP_ACK'  ].values)
        smaller['TCP_URG_sum'  ]=flagsum (smaller['TCP_URG'  ].values)
        smaller['TCP_ECE_sum'  ]=flagsum (smaller['TCP_ECE'  ].values)
        smaller['TCP_CWR_sum'  ]=flagsum (smaller['TCP_CWR'  ].values)
        #del smaller["dport"]        
        #del smaller["sport"]        
        #del smaller["ID"]


        smaller["TCP_FIN_ratio"]=smaller["TCP_FIN"]/(smaller["TCP_FIN_sum"]+10e-20)
        smaller["TCP_SYN_ratio"]=smaller["TCP_SYN"]/(smaller["TCP_SYN_sum"]+10e-20)
        smaller["TCP_RST_ratio"]=smaller["TCP_RST"]/(smaller["TCP_RST_sum"]+10e-20)
        smaller["TCP_PSH_ratio"]=smaller["TCP_PSH"]/(smaller["TCP_PSH_sum"]+10e-20)
        smaller["TCP_ACK_ratio"]=smaller["TCP_ACK"]/(smaller["TCP_ACK_sum"]+10e-20)
        smaller["TCP_URG_ratio"]=smaller["TCP_URG"]/(smaller["TCP_URG_sum"]+10e-20)
        smaller["TCP_ECE_ratio"]=smaller["TCP_ECE"]/(smaller["TCP_ECE_sum"]+10e-20)
        smaller["TCP_CWR_ratio"]=smaller["TCP_CWR"]/(smaller["TCP_CWR_sum"]+10e-20)
        col_list= ["TCP_FIN_sum","TCP_SYN_sum","TCP_RST_sum","TCP_PSH_sum","TCP_ACK_sum","TCP_URG_sum","TCP_ECE_sum","TCP_CWR_sum"]
        smaller['sum'] = smaller[col_list].sum(axis=1)
        smaller["TCP_FIN_SR"]=smaller["TCP_FIN"]/(smaller["sum"]+10e-20)
        smaller["TCP_SYN_SR"]=smaller["TCP_SYN"]/(smaller["sum"]+10e-20)
        smaller["TCP_RST_SR"]=smaller["TCP_RST"]/(smaller["sum"]+10e-20)
        smaller["TCP_PSH_SR"]=smaller["TCP_PSH"]/(smaller["sum"]+10e-20)
        smaller["TCP_ACK_SR"]=smaller["TCP_ACK"]/(smaller["sum"]+10e-20)
        smaller["TCP_URG_SR"]=smaller["TCP_URG"]/(smaller["sum"]+10e-20)
        smaller["TCP_ECE_SR"]=smaller["TCP_ECE"]/(smaller["sum"]+10e-20)
        smaller["TCP_CWR_SR"]=smaller["TCP_CWR"]/(smaller["sum"]+10e-20)






        if flag:
            smaller.to_csv(j[13:],header=True,index=False)
            flag=0
        else:
            smaller.to_csv(j[13:],header=False,index=False,mode="a")



    df=pd.read_csv(j[13:])
    features=df.columns
    count=[]
    bos=[]

    #for ii,i in enumerate (features):
        #a=pd.DataFrame(df.groupby(i).size())
        #if a.shape[0]>1:
            #print(a)
           # count.append(i)
            #print("________________________________________________________________\n\n")
        #else:
            #bos.append(i)
    df=df.sort_values(by='Unnamed: 0')

    y=df["Label"]
    #for i in bos:
        #print(i)
        #del df[i]
    del df["Label"]
    del df['Unnamed: 0']
    df["Label"]=y
    last_name=f'./{outputfolder}/last_EW_{j[13:]}'
    make_smaller=True
    if make_smaller:
        attack=df[df["Label"]==1]
        normal=df[df["Label"]==0]
        if len(attack)>10000:
            attack=attack.sample(n=10000,  random_state=42) 
        if len(normal)>10000:
            normal=normal.sample(n=10000,  random_state=42) 
        df = pd.concat([normal,attack])
        df.to_csv(last_name)   
    else:
        df.to_csv(last_name,index=None)
    print("-------------------------------------------------------------------------------")


Active Wiretap_FE_test.csv


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:00<00:00, 60.70s/it]


-------------------------------------------------------------------------------
Active Wiretap_FE_train.csv


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [01:53<00:00, 113.16s/it]


-------------------------------------------------------------------------------
ARP MitM_FE_test.csv


...

-------------------------------------------------------------------------------
Video Injection_FE_test.csv


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:03<00:00, 63.04s/it]


-------------------------------------------------------------------------------
Video Injection_FE_train.csv


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:01<00:00, 61.74s/it]


-------------------------------------------------------------------------------


In [ ]:
import winsound
duration = 1000  # milliseconds
freq = 740  # Hz
winsound.Beep(freq, duration)